# Torchmoji cpu-based chain thaw fine-tuning

Set runtime accelerator to 'None'

code based on: https://github.com/huggingface/torchMoji/blob/master/examples/finetune_insults_chain-thaw.py

# Installs

In [1]:
!pip install emoji==0.6.0
!pip install numpy==1.21.5
!pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 torchaudio==0.10.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install scipy==1.5.2

Looking in links: https://download.pytorch.org/whl/torch_stable.html


# **Restart the runtime after installs!**

# Clone torchmoji repo
This is a modified version of the original Torchmoji that works with pytorch 1.10. I forked this to my github in case the modder removes it.

original: https://github.com/huggingface/torchMoji
modded: https://github.com/cw75/torchMoji

In [2]:
!git clone https://github.com/droesler/torchMoji

fatal: destination path 'torchMoji' already exists and is not an empty directory.


In [2]:
# change to torchMoji directory to run scripts from
import os
os.chdir('torchMoji')

# Download torchmoji pre-trained weights (Type 'y' when prompted)
These are the original huggingface pre-trained weights

In [4]:
!python3 scripts/download_weights.py

About to download the pretrained weights file from https://www.dropbox.com/s/q8lax9ary32c7t9/pytorch_model.bin?dl=0#
The size of the file is roughly 85MB. Continue? [y/n]
y
Downloading...
Running system call: wget https://www.dropbox.com/s/q8lax9ary32c7t9/pytorch_model.bin?dl=0# -O /content/torchMoji/model/pytorch_model.bin
--2022-04-30 23:36:47--  https://www.dropbox.com/s/q8lax9ary32c7t9/pytorch_model.bin?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/q8lax9ary32c7t9/pytorch_model.bin [following]
--2022-04-30 23:36:48--  https://www.dropbox.com/s/raw/q8lax9ary32c7t9/pytorch_model.bin
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc072b57be3aa5b79eba2e29a66e.dl.dropboxusercontent.com/cd/0/inline/BkZRmYU71g7mq

In [3]:
import emoji
import pandas as pd

In [4]:
un_processed = pd.read_pickle('/content/balanced_train_En.pkl')

In [5]:
un_processed.at[18, 'texts']

'I love drawing feets 👹👹👹\r\nMust draw more feet 👹✨'

# Remove emoji from train data

In [6]:
for index, row in un_processed.iterrows():
  un_processed.at[index, 'texts'] = emoji.get_emoji_regexp().sub(r'', row['texts'])

In [7]:
un_processed.at[18, 'texts']

'I love drawing feets \r\nMust draw more feet '

# Function from torchmoji github:

In [8]:
"""Finetuning example.
Trains the torchMoji model on the kaggle insults dataset, using the 'chain-thaw'
finetuning method and the accuracy metric. See the blog post at
https://medium.com/@bjarkefelbo/what-can-we-learn-from-emojis-6beb165a5ea0
for more information. Note that results may differ a bit due to slight
changes in preprocessing and train/val/test split.
The 'chain-thaw' method does the following:
0) Load all weights except for the softmax layer. Extend the embedding layer if
   necessary, initialising the new weights with random values.
1) Freeze every layer except the last (softmax) layer and train it.
2) Freeze every layer except the first layer and train it.
3) Freeze every layer except the second etc., until the second last layer.
4) Unfreeze all layers and train entire model.
"""

from __future__ import print_function
# import example_helper
import json
from torchmoji.model_def import torchmoji_transfer
from torchmoji.global_variables import PRETRAINED_PATH
from torchmoji.finetuning import (
     load_benchmark,
     finetune)


DATASET_PATH = '/content/balanced_train_En.pkl'
nb_classes = 2

with open('/content/torchMoji/model/vocabulary.json', 'r') as f:
    vocab = json.load(f)

# Load dataset. Extend the existing vocabulary with up to 10000 tokens from
# the training dataset.
data = load_benchmark(DATASET_PATH, vocab, extend_with=10000)

# Set up model and finetune. Note that we have to extend the embedding layer
# with the number of tokens added to the vocabulary.
model = torchmoji_transfer(nb_classes, PRETRAINED_PATH, extend_embedding=data['added'])
print(model)
model, acc = finetune(model, data['texts'], data['labels'], nb_classes,
                      data['batch_size'], method='chain-thaw')
print('Acc: {}'.format(acc))

/content/torchMoji/torchmoji/model_def.py:163: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.embed.weight.data, a=-0.5, b=0.5)
/content/torchMoji/torchmoji/model_def.py:165: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(t)
/content/torchMoji/torchmoji/model_def.py:167: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  nn.init.orthogonal(t)
/content/torchMoji/torchmoji/model_def.py:169: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(t, 0)
/content/torchMoji/torchmoji/model_def.py:171: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.output_layer[0].weight.data)


Loading weights for embed.weight
Loading weights for lstm_0.weight_ih_l0
Loading weights for lstm_0.weight_hh_l0
Loading weights for lstm_0.bias_ih_l0
Loading weights for lstm_0.bias_hh_l0
Loading weights for lstm_0.weight_ih_l0_reverse
Loading weights for lstm_0.weight_hh_l0_reverse
Loading weights for lstm_0.bias_ih_l0_reverse
Loading weights for lstm_0.bias_hh_l0_reverse
Loading weights for lstm_1.weight_ih_l0
Loading weights for lstm_1.weight_hh_l0
Loading weights for lstm_1.bias_ih_l0
Loading weights for lstm_1.bias_hh_l0
Loading weights for lstm_1.weight_ih_l0_reverse
Loading weights for lstm_1.weight_hh_l0_reverse
Loading weights for lstm_1.bias_ih_l0_reverse
Loading weights for lstm_1.bias_hh_l0_reverse
Loading weights for attention_layer.attention_vector
Ignoring weights for output_layer.0.weight
Ignoring weights for output_layer.0.bias
TorchMoji(
  (embed): Embedding(50000, 256)
  (embed_dropout): Dropout2d(p=0.1, inplace=False)
  (lstm_0): LSTMHardSigmoid(256, 512, batch_fir

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


original val loss 0.69211817


/content/torchMoji/torchmoji/finetuning.py:536: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  clip_grad_norm(model.parameters(), 1)


== Epoch 0 step 0 train loss 0.6933414 train acc 124.808
== Epoch 0 step 1 train loss 0.6924465 train acc 125.104
== Epoch 0 step 2 train loss 0.69423777 train acc 125.576
== Epoch 0 step 3 train loss 0.6946879 train acc 125.416
== Epoch 0 step 4 train loss 0.69191885 train acc 124.92
== Epoch 0 step 5 train loss 0.6929087 train acc 125.096
== Epoch 0 step 6 train loss 0.6923277 train acc 125.44
== Epoch 0 step 7 train loss 0.69161093 train acc 125.0
== Epoch 0 step 8 train loss 0.6914707 train acc 124.952
== Epoch 0 step 9 train loss 0.6897964 train acc 127.704
== Epoch 0 step 10 train loss 0.68934363 train acc 123.88
== Epoch 0 step 11 train loss 0.68763876 train acc 124.328
== Epoch 0 step 12 train loss 0.68754464 train acc 129.368
== Epoch 0 step 13 train loss 0.6827752 train acc 124.208
== Epoch 0 step 14 train loss 0.6871491 train acc 124.64
== Epoch 0 step 15 train loss 0.69021946 train acc 124.392
== Epoch 0 step 16 train loss 0.68908525 train acc 125.0
== Epoch 0 step 17 train

In [9]:
# torch.save(bert_classifier, 'test_model.pth')
# /tmp/tmpju51ra8m/torchmoji-checkpoint-a798265f-07a8-4a11-ae81-3ce26a74b936.bin
from google.colab import files
files.download('/tmp/tmpju51ra8m/torchmoji-checkpoint-a798265f-07a8-4a11-ae81-3ce26a74b936.bin') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>